In [1]:
import pandas as pd
import numpy as np

In [2]:
data_full = pd.read_csv("C:/Users/rafal/Desktop/DS/Merceri price/train.tsv", sep = '\t')

In [3]:
data = data_full.sample(100000)

Używam tylko 100 000 obserwacji.

In [4]:
data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
1082956,1082956,COACH jewelry case (round) Leather,1,Women/Jewelry/Necklaces,Coach,29.0,1,COACH jewelry zippered CASE / jewelry box (rou...
511824,511824,Freeship NWT Wacoal Bra 38DD,1,Women/Underwear/Bras,NaN,40.0,1,"When you touch this bra, you'll be amazed at h..."
762614,762614,Cute Shirt,3,Women/Tops & Blouses/T-Shirts,NaN,6.0,1,Wrinkled from storage. Very light weight. Ship...
244571,244571,*reserved*,1,Women/Dresses/Full-Length,Rue21,12.0,0,Adorable cream and mint maxi style dress with ...
405303,405303,Lane Bryant 18/20 plus 1x gray sweater,2,Women/Tops & Blouses/Blouse,Lane Bryant,14.0,0,"Like new, worn once! Lane Bryant sweater, size..."


In [5]:
pv = pd.pivot_table(data, values = ['price'],index = ['brand_name'], aggfunc = ['count', np.mean, np.median])

In [6]:
pv.columns = pv.columns.get_level_values(0)

In [7]:
pv_sorted = pv.sort_values(by = ['count'], ascending = False)

In [8]:
data.describe()

,train_id,item_condition_id,price,shipping
count,1.000000e+05,100000.000000,100000.000000,100000.000000
mean,7.422840e+05,1.901270,26.701765,0.449560
std,4.279479e+05,0.902281,38.940803,0.497452
min,1.300000e+01,1.000000,0.000000,0.000000
25%,3.726665e+05,1.000000,10.000000,0.000000
50%,7.410960e+05,2.000000,17.000000,0.000000
75%,1.112512e+06,3.000000,29.000000,1.000000
max,1.482490e+06,5.000000,2004.000000,1.000000


## Preprocessing the category variable

In [ ]:
def StringToColumns(data, column_name, sep, split_limit, fill_na = "No_category"):
    categories = data[column_name].str.split(sep, split_limit, expand = True)
    categories = categories.drop(columns = 3, axis = 1)
    categories = categories.fillna(value = fill_na)
    data_cats = pd.concat([data, categories], axis = 1)
    data_features = data_cats.drop(columns = [column_name], axis = 1)
    return data_features

In [9]:
categories = data['category_name'].str.split('/', 3, expand = True)

In [10]:
categories.isna().sum()

0      435
1      435
2      435
3    99707
dtype: int64

Wyrzucam 4 poziom kategorii i uzupełniamy NA.

In [11]:
categories = categories.drop(columns = 3, axis = 1)
categories = categories.fillna(value = 'No_category')

In [12]:
categories.head()

,0,1,2
1151752,Beauty,Makeup,Makeup Sets
1438781,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins"
1450193,Women,Shoes,Athletic
237510,Women,Shoes,Boots
448932,Electronics,Cell Phones & Accessories,Cell Phones & Smartphones


In [13]:
categories.isna().sum()

0    0
1    0
2    0
dtype: int64

In [14]:
data_cats = pd.concat([data, categories], axis = 1)

In [15]:
data_cats.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,0,1,2
1151752,1151752,Beauty/health/skincare lot of items NEW,1,Beauty/Makeup/Makeup Sets,Burt's Bees,20.0,0,"Lot of beauty, health, hair and skincare items...",Beauty,Makeup,Makeup Sets
1438781,1438781,2 IPHONE 7 PLUS Supreme Cases,1,"Electronics/Cell Phones & Accessories/Cases, C...",Apple,8.0,1,2 IPHONE 7PLUS SUPREME CASES. Also clik on my ...,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins"
1450193,1450193,Nike Flyknit Air Max,2,Women/Shoes/Athletic,Nike,106.0,0,Authentic Nike Flyknit Air Max running sneaker...,Women,Shoes,Athletic
237510,237510,Brand-new boots,2,Women/Shoes/Boots,NaN,9.0,0,Brand-new boots. Only used once! Size 8/9,Women,Shoes,Boots
448932,448932,APPLE WATCH BAND 42MM AUTHENIC,1,Electronics/Cell Phones & Accessories/Cell Pho...,Apple,28.0,1,MIDNIGHT BLUE NEW NEW NEW NEW!!! AUTHENTIC!,Electronics,Cell Phones & Accessories,Cell Phones & Smartphones


In [16]:
y = data_cats.loc[:, 'price']
data_features = data_cats.drop(columns = ['price', 'category_name'], axis = 1)

In [17]:
data_features.loc[:, 'brand_name'] = data_features.loc[:, 'brand_name'].fillna(value = 'No_brand_name')

In [18]:
data_features.head()

,train_id,name,item_condition_id,brand_name,shipping,item_description,0,1,2
1151752,1151752,Beauty/health/skincare lot of items NEW,1,Burt's Bees,0,"Lot of beauty, health, hair and skincare items...",Beauty,Makeup,Makeup Sets
1438781,1438781,2 IPHONE 7 PLUS Supreme Cases,1,Apple,1,2 IPHONE 7PLUS SUPREME CASES. Also clik on my ...,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins"
1450193,1450193,Nike Flyknit Air Max,2,Nike,0,Authentic Nike Flyknit Air Max running sneaker...,Women,Shoes,Athletic
237510,237510,Brand-new boots,2,No_brand_name,0,Brand-new boots. Only used once! Size 8/9,Women,Shoes,Boots
448932,448932,APPLE WATCH BAND 42MM AUTHENIC,1,Apple,1,MIDNIGHT BLUE NEW NEW NEW NEW!!! AUTHENTIC!,Electronics,Cell Phones & Accessories,Cell Phones & Smartphones


## Preprocessing the name variable

In [19]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
corpus = []
for i in range(0, len(data)):
    name = re.sub('[^a-zA-Z]', ' ', data_features.iloc[i, 1])
    name = name.lower()
    name = name.split()
    ps = PorterStemmer()
    name = [ps.stem(word) for word in name if not word in set(stopwords.words('english'))]
    name = ' '.join(name)
    corpus.append(name)

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x00000180DD810B70>
Traceback (most recent call last):
  File "C:\Users\rafal\Anaconda3\lib\site-packages\nltk\data.py", line 1281, in __del__
    self.close()
  File "C:\Users\rafal\Anaconda3\lib\site-packages\nltk\data.py", line 1310, in close
    self.stream.close()
KeyboardInterrupt


In [21]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)
name_features = cv.fit_transform(corpus)

In [22]:
df_name_features = pd.DataFrame(name_features.toarray())

## Preprocessing the brands variable

In [ ]:
def SelectedLevelsToColumns_count(data, target, categorical, percentage=0.001):
    pivot_brands = pd.pivot_table(data, values = target, index = ['brand_name'], aggfunc = ['count'])
    pivot_brands.columns = pivot_brands.columns.get_level_values(0)
    pv_sorted = pivot_brands.sort_values(by = ['count'], ascending = False)
    selected_brands = pv_sorted[pv_sorted['count']>0.001*len(data)]
    brand_list = selected_brands.index
    brand_list = np.array(brand_list)
    ohe = OneHotEncoder()
    cat_data = pd.DataFrame(data['brand_name'])
    cat_data.fillna(value = 'no_brand_name', inplace = True)
    data['new_brand_name'] = np.where(data['brand_name'].isin(brand_list), data['brand_name'], 'other')
    categories = ohe.fit_transform(pd.DataFrame(data['new_brand_name']))
    data.drop(columns=['new_brand_name', 'brand_name'], axis = 1, inplace = True)
    data.reset_index(drop = True, inplace = True)
    data_final = pd.concat([data, pd.DataFrame(categories.toarray())], axis = 1)
    return data_final, brand_list

## Preprocessing the item description variable

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
def WordsToColumns(data, max_columns, column_number):
    corpus = []
    for i in range(0, len(data)):
        name = re.sub('[^a-zA-Z]', ' ', data.iloc[i, column_number])
        name = name.lower()
        name = name.split()
        ps = PorterStemmer()
        name = [ps.stem(word) for word in name if not word in set(stopwords.words('english'))]
        name = ' '.join(name)
        corpus.append(name)    
    # Creating the Bag of Words model
    cv = CountVectorizer(max_features = max_columns)
    name_features = cv.fit_transform(corpus)
    df_name_features = pd.DataFrame(name_features.toarray())
    return df_name_features

In [18]:
data['item_description'].fillna(value = "", inplace = True)

In [20]:
desc_prepro = WordsToColumns(data = data, max_columns=10000, column_number=7)

In [22]:
desc_prepro.to_csv('description_preprocessed.csv')

In [74]:
pivot_brands = pd.pivot_table(data, values = ['price'],index = ['brand_name'], aggfunc = ['count'])

In [75]:
pivot_brands.head()

,count
,price
brand_name,
% Pure,6
3.1 Phillip Lim,2
47 Brand,3
5.11 Tactical,2
5th & Ocean,1


In [77]:
pivot_brands.columns = pivot_brands.columns.get_level_values(0)

In [78]:
pv_sorted = pivot_brands.sort_values(by = ['count'], ascending = False)

In [79]:
selected_brands = pv_sorted[pv_sorted['count']>0.001*len(data)]

In [115]:
brand_list = selected_brands.index

AttributeError: 'Index' object has no attribute 'toarray'

In [117]:
brand_list = np.array(brand_list)

In [95]:
from sklearn.preprocessing import OneHotEncoder

In [120]:
ohe = OneHotEncoder()

In [103]:
cat_data = pd.DataFrame(data['brand_name'])

In [106]:
cat_data.fillna(value = 'no_brand_name', inplace = True)

In [126]:
data['new_brand_name'] = np.where(data['brand_name'].isin(brand_list), data['brand_name'], 'other')

In [130]:
categories = ohe.fit_transform(pd.DataFrame(data['new_brand_name']))

In [135]:
data.drop(columns=['new_brand_name', 'brand_name'], axis = 1, inplace = True)

In [139]:
data.reset_index(drop = True, inplace = True)
data_final = pd.concat([data, pd.DataFrame(categories.toarray())], axis = 1)

In [140]:
data_final.head()

,train_id,name,item_condition_id,category_name,price,shipping,item_description,0,1,2,...,97,98,99,100,101,102,103,104,105,106
0,1151752,Beauty/health/skincare lot of items NEW,1,Beauty/Makeup/Makeup Sets,20.0,0,"Lot of beauty, health, hair and skincare items...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1438781,2 IPHONE 7 PLUS Supreme Cases,1,"Electronics/Cell Phones & Accessories/Cases, C...",8.0,1,2 IPHONE 7PLUS SUPREME CASES. Also clik on my ...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1450193,Nike Flyknit Air Max,2,Women/Shoes/Athletic,106.0,0,Authentic Nike Flyknit Air Max running sneaker...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,237510,Brand-new boots,2,Women/Shoes/Boots,9.0,0,Brand-new boots. Only used once! Size 8/9,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,448932,APPLE WATCH BAND 42MM AUTHENIC,1,Electronics/Cell Phones & Accessories/Cell Pho...,28.0,1,MIDNIGHT BLUE NEW NEW NEW NEW!!! AUTHENTIC!,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
data_features.reset_index(drop = True, inplace = True)
data_c_name = pd.concat([data_features, df_name_features], axis = 1)

In [1]:
data.head()

NameError: name 'data' is not defined

In [25]:
data_c_name.drop(columns = 'name', inplace=True)

In [27]:
len(data_c_name['brand_name'].unique())

2050

In [28]:
brands = pd.pivot_table(data = data_cats, values = 'price', aggfunc= 'count', index = 'brand_name')

In [29]:
brands.sort_values(by = 'price', ascending=False, inplace=True)
brands.rename(columns = {'price':'count'}, inplace=True)

In [30]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [31]:
#preprocess = ColumnTransformer([('brand_name', OneHotEncoder(), 'brand_name')])

In [32]:
ohe = OneHotEncoder(categorical_features=[2])

In [33]:
#brand_names = ohe.fit_transform(data_c_name)

In [34]:
#df4 = df3[df3.columns[df3.columns.isin(brands.iloc[0:172, 0])]]

In [35]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

In [36]:
#preprocess = make_column_transformer